In [1]:
# Sean Dougherty
# 03/15/2022
# this file reads in the output csv filed from agn_merger.py and does some preliminary visualization
# will later become a .py file for sharing and convinience

In [2]:
# at the end of csv processing, we want a df with bins as columns and iterations as rows
# we want 3 of these: for pair AGN fraction, iso AGN fraction, and enhancement

In [7]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

## Checking pair and control galaxy mass and redshift distributions

## Calculate AGN fractions

In [14]:
# create bins
max_dist = 100
n_bins = 10
bins, bin_all_pairfrac, bin_all_isofrac = make_bins(max_dist=max_dist, n_bins=n_bins)

# define X-ray luminosity AGN limit
LX_AGN = 42

# create dictionaries to store AGN fractions
bin_AGN = {}
bin_pairfrac = {}
bin_isofrac = {}

# read through all the iteration files
data_path = '/nobackup/c1029594/CANDELS_AGN_merger_data/agn_merger_output/'
for file in os.listdir(data_path):
    print(file)
#     it_df = pd.read_csv(file)
    
#     # separate pairs into bins
#     for rnge in bins:
#         # get bin ranges from bins dictionary entries
#         info = all_bins[rnge].split('-')
#         low = float(info[0])
#         high = float(info[1])
        
#         # sort into bins
#         bin_df = it_df[ (it_df['kpc_sep'] > low) & (it_df['kpc_sep'] <= high) ]
        
#         # calculate the AGN fraction in that bin and append it to same calculation accross iteration
#         bin_pairfrac['AGNfrac_in_'+rnge] = ((  (bin_df[ (bin_df['prime_drawn_LX'] > 42) ]).count() +
#                                              (bin_df[ (bin_df['partner_drawn_LX'] > 42) ]).count()
#                                             ) /
#                                             (  bin_df['prime_drawn_LX'].count() +
#                                              bin_df['partner_drawn_LX'].count()
#                                             ))
#         bin_all_pairfrac['all_AGNfrac_in_'+rnge].append(bin_pairfrac['AGNfrac_in_'+rnge])

        
#         bin_isofrac['AGNfrac_in_'+rnge] = ((  (bin_df[ (bin_df['prime_control1_drawn_LX'] > 42).count() ]) +
#                                             (bin_df[ (bin_df['prime_control2_drawn_LX'] > 42) ]).count() +
#                                             (bin_df[ (bin_df['partner_control1_drawn_LX'] > 42) ]).count() +
#                                             (bin_df[ (bin_df['partner_control2_drawn_LX'] > 42) ]).count() 
#                                            ) /
#                                             (  bin_df['prime_control1_drawn_LX'].count() +
#                                              bin_df['prime_control2_drawn_LX'].count() +
#                                              bin_df['partner_control1_drawn_LX'].count() +
#                                              bin_df['partner_control2_drawn_LX'].count() ))
#         bin_all_isofrac['all_AGNfrac_in_'+rnge].append(bin_isofrac['AGNfrac_in_'+rnge])
        

zphot_29
zphot_23
zphot_0
zphot_11
zphot_30
zphot_47
zphot_40
zphot_37
zphot_16
zphot_7
zphot_24
zphot_9
zphot_3
zphot_20
zphot_12
zphot_18
zphot_39
zphot_44
zphot_33
zphot_49
zphot_34
zphot_43
zphot_15
zphot_27
zphot_4
zphot_17
zphot_6
zphot_25
zphot_36
zphot_41
zphot_46
zphot_31
zphot_22
zphot_28
zphot_1
zphot_10
zphot_14
zphot_26
zphot_5
zphot_42
zphot_35
zphot_48
zphot_32
zphot_45
zphot_38
zphot_2
zphot_8
zphot_21
zphot_19
zphot_13


In [11]:
def make_bins(max_dist=100, n_bins=10):

    bin_all_AGNfrac = {}
    bin_all_isofrac = {}

    # create bin sizes
    all_bins = {}
    for i in range(0, n_bins):
        # define limits for each bin
        all_bins['bin'+str(i)] = str((max_dist/n_bins)*i)+'-'+str((max_dist/n_bins)+(max_dist/n_bins)*i)
        # define empty lists to later append AGN fractions to
        bin_all_AGNfrac['all_AGNfrac_in_bin'+str(i)] = []
        bin_all_isofrac['all_isofrac_in_bin'+str(i)] = []
    
    return all_bins, bin_all_AGNfrac, bin_all_isofrac

In [ ]:
# create dataframes from the dictionaries:
pAGNf_df = pd.from_dict(bin_all_pairfrac)
iAGNf_df = pd.from_dict(bin_all_isofrac)

In [ ]:
# plot distributions
pAGNf_df.plt.hist(bins=10, histtype='step')